## 0. Libraries 

In [2]:
import pandas as pd
import matplotlib
import numpy as np  
from googletrans import Translator
from textblob import TextBlob
from deep_translator import GoogleTranslator
import time
from textblob import TextBlob

ModuleNotFoundError: No module named 'deep_translator'

## 1. Loading Data

In [1]:
# Load the clean data
df = pd.read_csv('data/clean/clean_df.csv')

NameError: name 'pd' is not defined

## 2. Data Understandig 

- Numbers of interviews 

- 2005 - 815
- 2007 - 797
- 2009 - 827 
- 2011 - 1352
- 2013 - 1248
- 2015 - 1102
- 2017 - 1116
- 2019 - 2400
- 2021 - 3000
- 2023 - 21787



In [ ]:
df.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1779 entries, 0 to 1778
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   bloco       1779 non-null   object
 1   question    1779 non-null   object
 2   year        1779 non-null   int64 
 3   answer      1779 non-null   object
 4   percentage  1779 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 69.6+ KB


In [4]:
df.duplicated().sum()  # Check for duplicates

np.int64(0)

In [5]:
df.head()   # Check the first few rows of the data

,bloco,question,year,answer,percentage
0,Percepção geral sobre violência contra as mulh...,"De forma geral, você acha que as mulheres são ...",2005,Sim,8
1,Percepção geral sobre violência contra as mulh...,"De forma geral, você acha que as mulheres são ...",2005,Às vezes,35
2,Percepção geral sobre violência contra as mulh...,"De forma geral, você acha que as mulheres são ...",2005,Não,56
3,Percepção geral sobre violência contra as mulh...,"De forma geral, você acha que as mulheres são ...",2005,NS/NR,0
4,Percepção geral sobre violência contra as mulh...,"De forma geral, você acha que as mulheres são ...",2007,Sim,8


## 3. Data Transformation 

In [6]:
# Define the function for translation
def translate_text(text, source_lang="pt", target_lang="en"):
    """Translates text using GoogleTranslator from deep-translator."""
    try:
        if pd.notna(text):  
            return GoogleTranslator(source=source_lang, target=target_lang).translate(text)
        return text
    except Exception as e:
        print(f"Translation failed for '{text}': {e}")
        return text  

# Ensure profile_df exists
if "df1" not in locals():
    df1 = df.copy()  

# Apply translation to relevant columns
df1["category_translated"] = df["bloco"].apply(translate_text)
df1["question_translated"] = df["question"].apply(translate_text)
df1["answer_translated"] = df["answer"].apply(translate_text)

In [9]:

# Dictionary with the number of interviews per year
interviews_per_year = {
    2005: 815,
    2007: 797,
    2009: 827,
    2011: 1352,
    2013: 1248,
    2015: 1102,
    2017: 1116,
    2019: 2400,
    2021: 3000,
    2023: 21787
}

# Add a new column calculating the total number of answers for each row
df1["total_answers"] = df1.apply(lambda row: int((row["percentage"] / 100) * interviews_per_year.get(row["year"], 1)), axis=1)

In [11]:
# Save the translated DataFrame
df1.to_csv("data/clean/translated_df.csv", index=False)

In [13]:
df2 = df1.copy()
df2.drop(columns=["bloco", "question", "answer"], inplace=True)
df2.rename(columns={"category_translated": "category", "question_translated": "question", "answer_translated": "answer"}, inplace=True)
df2[['year', 'category', 'question', 'answer', 'percentage']]
df2["year"] = df2["year"].astype(str)

# Create a new DataFrame with selected columns from df2
df3 = df2[['year', 'category', 'question', 'answer', 'percentage', 'total_answers']]
df3

,year,category,question,answer,percentage,total_answers
0,2005,General perception of violence against women,"In general, do you think women are treated wit...",Yes,8,65
1,2005,General perception of violence against women,"In general, do you think women are treated wit...",Sometimes,35,285
2,2005,General perception of violence against women,"In general, do you think women are treated wit...",No,56,456
3,2005,General perception of violence against women,"In general, do you think women are treated wit...",NS/NR,0,0
4,2007,General perception of violence against women,"In general, do you think women are treated wit...",Yes,8,63
...,...,...,...,...,...,...
1774,2023,Occupation profile,"Your individual income, not counting on the he...",Other,9,1960
1775,2023,Occupation profile,Is your individual income enough to keep you a...,Yes,37,8061
1776,2023,Occupation profile,Is your individual income enough to keep you a...,No,32,6971
1777,2023,Occupation profile,Is your individual income enough to keep you a...,There is no income,30,6536


In [14]:
# Save the final DataFrame
df3.to_csv("data/clean/final_translated_df.csv", index=False)